In [2]:
import socket
import pickle
import random

In [3]:
def get_control_bits(word_length):
    control_bits = []
    bit = 1
    while bit < word_length + len(control_bits):
        if not bit & (bit - 1):
            control_bits.append(bit-1)
        bit += 1
    return control_bits



def string2bits(s = ''):
    return ''.join([bin(ord(i))[2:].zfill(8) for i in s])



def get_words(data, word_length):
    while len(data) % word_length:
        data += '0'
    for i in range(len(data)):
        if not i % word_length:
            yield data[i:i+word_length]
            
            
            
def hamming_encode(word, control_bits):
    for i in control_bits:
        word = word[:i] + '0' + word[i:]
    pow2 = 1
    for control_bit in control_bits:
        mult = 0
        for i in range(len(word) - control_bit):
            if i % (pow2 * 2) >= pow2:
                continue
            if word[control_bit+i] == '1':
                mult ^= 1
        if mult == 1:
            word = word[:control_bit] + '1' + word[control_bit+1:]
        pow2 *= 2
    return word



def message_processing(message, word_length, control_bits):
    bits = string2bits(message)
    encoded_message = ''
    for word in get_words(bits, word_length):
        encoded_message += hamming_encode(word, control_bits)
    return encoded_message



def add_mistakes(data, word_length, mistakes_number=0):
    mistakes = []
    correctly = 0
    uncorrectly = 0
    message_with_mistakes = ''
    p_word = 0.4
    p_in_word = 0.05
    word_index = 0
    for word in get_words(data, word_length):
        correct = 1
        if random.random() < p_word:
            n = 0
            for i in range(len(word)):
                if random.random() < p_in_word:
                    correct = 0
                    s = '0'
                    if word[i] == '0':
                        s = '1'
                    word = word[:i] + s + word[i+1:]
                    n += 1
                    mistakes.append((word_index, i))
                if n == mistakes_number:
                    break
        if correct:
            correctly += 1
        else:
            uncorrectly += 1
        message_with_mistakes += word
        word_index += 1
    return message_with_mistakes, mistakes, correctly, uncorrectly

In [6]:
word_length = 15  
control_bits = get_control_bits(word_length)
host = "127.0.0.1"
port = 8
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((host, port))

message = input('Введите сообщение: ')
processed_message = message_processing(message, word_length, control_bits)
mistakes_number = int(input('Введите количество ошибок на слово: '))
message_with_mistakes, mistakes, correctly, uncorrectly = add_mistakes(processed_message, word_length+len(control_bits), mistakes_number)
sock.send(pickle.dumps((message_with_mistakes, word_length)))

mistakes_from_server, correctly_delivered, uncorrectly_delivered = pickle.loads(sock.recv(1024))
sock.close()

print()
print("Количество правильно доставленных слов: {} ({})".format(correctly_delivered, correctly))
print("Количество неправильно доставленных слов: {} ({})".format(uncorrectly_delivered, uncorrectly))
print()
print("Добавленные ошибки:")
ind = mistakes[0][0]
current_mistakes = []
for mistake in mistakes:
    if ind != mistake[0]:
        print(current_mistakes)
        current_mistakes = [mistake]
        ind = mistake[0]
    else:
        current_mistakes.append(mistake)
print(current_mistakes)
print("Найденные сервером ошибки:")
ind = mistakes_from_server[0][0]
current_mistakes = []
for mistake in mistakes_from_server:
    if ind != mistake[0]:
        print(current_mistakes)
        current_mistakes = [mistake]
        ind = mistake[0]
    else:
        current_mistakes.append(mistake)
print(current_mistakes)

Введите сообщение: dfvgbhjmk,l.;
Введите количество ошибок на слово: 10

Количество правильно доставленных слов: 6 (6)
Количество неправильно доставленных слов: 1 (1)

Добавленные ошибки:
[(5, 11), (5, 13)]
Найденные сервером ошибки:
[(5, 1)]


In [7]:
print("Количество правильно доставленных слов: {} ({})".format(5, 3))

Количество правильно доставленных слов: 5 (3)


In [12]:
print("Добавленные ошибки:")
ind = mistakes[0][0]
current_mistakes = []
for mistake in mistakes:
    if ind != mistake[0]:
        print(current_mistakes)
        current_mistakes = []
        ind = mistake[0]
    else:
        current_mistakes.append(mistake)
print(current_mistakes)
print("Найденные сервером ошибки:")
ind = mistakes_from_server[0][0]
current_mistakes = []
for mistake in mistakes_from_server:
    if ind != mistake[0]:
        print(current_mistakes)
        current_mistakes = []
        ind = mistake[0]
    else:
        current_mistakes.append(mistake)
print(current_mistakes)

Добавленные ошибки:
[(3, 12)]
[(6, 10)]
[(9, 9)]
[]
[(11, 7)]
[]
[]
[]
[]
[]
[(31, 18)]
[]
Найденные сервером ошибки:
[(3, 12)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [ ]:
print("Добавленные ошибки:")
ind = mistakes[0][0]
current_mistakes = []
for mistake in mistakes:
    if ind != mistake[0]:
        print(current_mistakes)
        current_mistakes = [mistake]
        ind = mistake[0]
    else:
        current_mistakes.append(mistake)
print(current_mistakes)
print("Найденные сервером ошибки:")
ind = mistakes_from_server[0][0]
current_mistakes = []
for mistake in mistakes_from_server:
    if ind != mistake[0]:
        print(current_mistakes)
        current_mistakes = [mistake]
        ind = mistake[0]
    else:
        current_mistakes.append(mistake)
print(current_mistakes)